In [18]:
!pip install PyMySQL

In [19]:
import sys
import pymysql
import datetime

import getNYT

In [23]:
try:
    conn = pymysql.connect(host='10.120.28.52',
                       user='gp3',
                       password='Group_03',
                       db='gp3',
                       charset='utf8',
                       autocommit=True,
                       cursorclass=pymysql.cursors.DictCursor)
    c= conn.cursor()
except:
    print('Cannot connect to MySQL server!')

In [24]:
articleList = getNYT.getArticleList(nDays=1)

for i in range(len(articleList)):
    print(articleList[i]['url'])

https://www.nytimes.com/2017/04/14/world/africa/chibok-girls-nigeria-boko-haram.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/europe/paris-tourism.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/europe/trump-tower-neighbor-europe-populists-orban-wilders-le-pen.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/africa/congo-un-investigators-murder-arrests.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/asia/south-korea-election-ahn-cheol-soo.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/asia/bomb-afghanistan.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/middleeast/syria-towns-sieges-population-transfers-assad.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/middleeast/bana-al-abed-syria-aleppo-twitter-book-deal.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/europe/turkey-erdogan-democracy-referendum.html?ref=todayspaper
https://www.nytimes.com/2017/04/14/world/europe/turkis

In [27]:
for i in range(len(articleList)):
    try:
        c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",articleList[i])
    except:
        print('error occurs',sys.exc_info())

error occurs (<class 'pymysql.err.IntegrityError'>, IntegrityError(1062, "Duplicate entry 'https://www.nytimes.com/2017/04/14/world/africa/chibok-girls-nig' for key 'PRIMARY'"), <traceback object at 0x00000000080E2048>)
error occurs (<class 'pymysql.err.IntegrityError'>, IntegrityError(1062, "Duplicate entry 'https://www.nytimes.com/2017/04/14/world/europe/paris-tourism.ht' for key 'PRIMARY'"), <traceback object at 0x00000000080E20C8>)
error occurs (<class 'pymysql.err.IntegrityError'>, IntegrityError(1062, "Duplicate entry 'https://www.nytimes.com/2017/04/14/world/europe/trump-tower-neig' for key 'PRIMARY'"), <traceback object at 0x00000000080E2088>)
error occurs (<class 'pymysql.err.IntegrityError'>, IntegrityError(1062, "Duplicate entry 'https://www.nytimes.com/2017/04/14/world/africa/congo-un-investi' for key 'PRIMARY'"), <traceback object at 0x00000000080E2D08>)
error occurs (<class 'pymysql.err.IntegrityError'>, IntegrityError(1062, "Duplicate entry 'https://www.nytimes.com/2017/